# Policy Bot Dataset Builder

This notebook scrapes Airbnb help pages and generates policy scenarios for training.


## Environment Setup

Set HuggingFace token for dataset uploads.


In [1]:
%env HF_TOKEN=your-huggingface-token-here

env: HF_TOKEN=your-huggingface-token-here


## Web Scraping

Recursively scrape Airbnb help pages to collect policy documentation.


In [ ]:
from parsing.parse import recursive_scrape  # a function I made to scrape a website and links on the page recursively
from tqdm import tqdm
import pandas as pd

urls = [
    'https://www.airbnb.com'  # Airbnb's website
]

depth, max_links_per_page = 5, 10
max_possible_articles = sum([max_links_per_page**d for d in range(0, depth+1)])
print(f"Maximum total number of articles possible (theoretical, with max_depth={depth}, max_links_per_page={max_links_per_page}): {max_possible_articles}")

Maximum total number of articles possible (theoretical, with max_depth=5, max_links_per_page=10): 111111


In [3]:
# Use recursive_scrape to get articles with metadata (title, content, etc.)
print("Testing recursive scraping with better parameters...")
scraping_results = recursive_scrape(start_urls=urls, max_depth=depth, max_links_per_page=max_links_per_page, base_domain="airbnb.com", delay=0.3)
print(f'Found {len(scraping_results)} articles')

# Convert to DataFrame
df = pd.DataFrame(scraping_results)
df.head()

INFO:parsing.parse:Scraping https://www.airbnb.com (depth: 0)


Testing recursive scraping with better parameters...


INFO:parsing.parse:Found 4 related links on https://www.airbnb.com
INFO:parsing.parse:Scraping https://www.airbnb.com/help/home (depth: 1)
INFO:parsing.parse:Found 4 related links on https://www.airbnb.com/help/home
INFO:parsing.parse:Scraping https://www.airbnb.com/help/article/2701/extenuating-circumstances-policy-and-the-coronavirus-covid19 (depth: 1)
INFO:parsing.parse:Found 10 related links on https://www.airbnb.com/help/article/2701/extenuating-circumstances-policy-and-the-coronavirus-covid19
INFO:parsing.parse:Scraping https://www.airbnb.com/help/responsible-hosting (depth: 1)
INFO:parsing.parse:Found 10 related links on https://www.airbnb.com/help/responsible-hosting
INFO:parsing.parse:Scraping https://www.airbnb.com/help/sale-share-opt-out (depth: 1)
INFO:parsing.parse:Found 7 related links on https://www.airbnb.com/help/sale-share-opt-out
INFO:parsing.parse:Scraping https://www.airbnb.com/help/all-topics (depth: 2)
INFO:parsing.parse:Found 6 related links on https://www.airbn

Found 1042 articles


,0,1,2,3,4
0,"(url, https://www.airbnb.com)","(title, Airbnb: Vacation Rentals, Cabins, Beac...","(content, \nAirbnb: Vacation Rentals, Cabins, ...","(links_found, [])","(depth, 0)"
1,"(url, https://www.airbnb.com/help/home)","(title, Customer Service - Airbnb Help Center)","(content, \nCustomer Service - Airbnb Help Cen...","(links_found, [])","(depth, 1)"
2,"(url, https://www.airbnb.com/help/article/2701...","(title, Major Disruptive Events Policy - Airbn...","(content, \nMajor Disruptive Events Policy - A...","(links_found, [])","(depth, 1)"
3,"(url, https://www.airbnb.com/help/responsible-...","(title, Local rules and regulations - Airbnb H...","(content, \nLocal rules and regulations - Airb...","(links_found, [])","(depth, 1)"
4,"(url, https://www.airbnb.com/help/sale-share-o...","(title, Your privacy choices - Airbnb Help Cen...","(content, \nYour privacy choices - Airbnb Help...","(links_found, [])","(depth, 1)"


## Data Processing

Convert scraped documents into structured DataFrame format.


In [4]:
documents = []
seen_urls = set()  # we have deduping in parse.py but just in case :)
for idx, result in enumerate(scraping_results):
    if result.url not in seen_urls:
        documents.append({
            'url': result.url,
            'title': result.title,
            'article': result.content,
        })
        seen_urls.add(result.url)

df = pd.DataFrame(documents)
print(f"Total documents: {len(df)}")
df.head()

Total documents: 1042


,url,title,article
0,https://www.airbnb.com,"Airbnb: Vacation Rentals, Cabins, Beach Houses...","\nAirbnb: Vacation Rentals, Cabins, Beach Hous..."
1,https://www.airbnb.com/help/home,Customer Service - Airbnb Help Center,\nCustomer Service - Airbnb Help CenterSkip to...
2,https://www.airbnb.com/help/article/2701/exten...,Major Disruptive Events Policy - Airbnb Help C...,\nMajor Disruptive Events Policy - Airbnb Help...
3,https://www.airbnb.com/help/responsible-hosting,Local rules and regulations - Airbnb Help Center,\nLocal rules and regulations - Airbnb Help Ce...
4,https://www.airbnb.com/help/sale-share-opt-out,Your privacy choices - Airbnb Help Center,\nYour privacy choices - Airbnb Help CenterSki...


## Scenario Generation

Generate synthetic customer scenarios and solutions from policy documents using AI.


In [ ]:
from parsing.parse import generate_scenarios_from_text
rows = []

In [14]:
for idx, document in tqdm(enumerate(documents), total=len(documents)):
    if idx < len(rows) - 1:
        continue

    try:
        scenarios = generate_scenarios_from_text(document['article'], k=1)
        for scenario in scenarios.scenarios:
            rows.append({
                'url': document['url'],
                'situation': scenario.situation,
                'solution': scenario.solution,
                'document_id': idx
            })
    except Exception as e:
        print(f"Error generating scenarios for document {idx}: {e}")
        continue


  0%|          | 0/1042 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 503 Service Temporarily Unavailable"
INFO:openai._base_client:Retrying request to /chat/completions in 0.410753 seconds
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 503 Service Temporarily Unavailable"
INFO:openai._base_client:Retrying request to /chat/completions in 0.813203 seconds
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 503 Service Temporarily Unavailable"
 27%|██▋       | 286/1042 [00:36<01:36,  7.84it/s]

Error generating scenarios for document 285: "Input to ChatPromptTemplate is missing variables {'k'}.  Expected: ['article_text', 'k'] Received: ['article_text']\nNote: if you intended {k} to be part of the string and not a variable, please escape it with double curly braces like: '{{k}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
 89%|████████▉ | 929/1042 [56:45<08:12,  4.35s/it]

Error generating scenarios for document 928: "Input to ChatPromptTemplate is missing variables {'k'}.  Expected: ['article_text', 'k'] Received: ['article_text']\nNote: if you intended {k} to be part of the string and not a variable, please escape it with double curly braces like: '{{k}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
100%|██████████| 1042/1042 [1:07:12<00:00,  3.87s/it]


In [15]:
df = pd.DataFrame(rows)
print(f'Number of rows: {len(df)}')
df['article'] = df['document_id'].apply(lambda x: documents[x]['article'])
df = df.dropna()
print(f'Number of rows after dropping na: {len(df)}')
df.head()

Number of rows: 1041
Number of rows after dropping na: 1041


,url,situation,solution,document_id,article
0,https://www.airbnb.com,A traveler is interested in renting a cottage ...,"According to the article, Airbnb offers a vari...",0,"\nAirbnb: Vacation Rentals, Cabins, Beach Hous..."
1,https://www.airbnb.com/help/home,A guest is concerned about a neighborhood issu...,"According to the Airbnb Help Center, there is ...",1,\nCustomer Service - Airbnb Help CenterSkip to...
2,https://www.airbnb.com/help/article/2701/exten...,A guest has a reservation for a beachfront pro...,According to Airbnb's Major Disruptive Events ...,2,\nMajor Disruptive Events Policy - Airbnb Help...
3,https://www.airbnb.com/help/responsible-hosting,A new host in Europe wants to know what regula...,"According to the article, Airbnb provides resp...",3,\nLocal rules and regulations - Airbnb Help Ce...
4,https://www.airbnb.com/help/sale-share-opt-out,A user from California is concerned that their...,"According to the article, if you reside in Cal...",4,\nYour privacy choices - Airbnb Help CenterSki...


## HuggingFace Upload

Upload the generated dataset to HuggingFace Hub for sharing.


In [16]:
# Upload DataFrame to HuggingFace Hub as a private dataset
from datasets import Dataset
from huggingface_hub import login, HfApi

# Convert DataFrame to HuggingFace Dataset
dataset = Dataset.from_pandas(df)
print(dataset)
# Define dataset name - change this to your preferred name
dataset_name = "airbnb-policy-scenarios"
username = "profoz"  # Replace with your actual HuggingFace username
repo_id = f"{username}/{dataset_name}"

Dataset({
    features: ['url', 'situation', 'solution', 'document_id', 'article'],
    num_rows: 1041
})


## Dataset Verification

Test loading the uploaded dataset to verify successful upload.


In [17]:
# Alternative: Check if dataset exists and update it if needed
from huggingface_hub import HfApi, repo_exists

api = HfApi()

# Check if dataset repository exists
if repo_exists(repo_id=repo_id, repo_type="dataset"):
    print(f"Dataset {repo_id} already exists. Updating...")
    dataset.push_to_hub(
        repo_id=repo_id,
        private=True,
        commit_message="Updated dataset with new scenarios",
        token=True
    )
    print("Dataset updated successfully!")

else:
    print(f"Creating new dataset: {repo_id}")
    dataset.push_to_hub(
        repo_id=repo_id,
        private=True,
        commit_message="Initial upload of Airbnb policy scenarios dataset",
        token=True
    )
    print("New dataset created successfully!")

# Show how to load the dataset later
print("\n" + "="*50)
print("To load this dataset later, use:")
print(f"from datasets import load_dataset")
print(f"dataset = load_dataset('{repo_id}')")
print("="*50)


Dataset profoz/airbnb-policy-scenarios already exists. Updating...


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Dataset updated successfully!

To load this dataset later, use:
from datasets import load_dataset
dataset = load_dataset('profoz/airbnb-policy-scenarios')
